# 0. Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm
import classifiers
import importlib
import pickle

# 1. Data Load

In [15]:
reactome = np.array(pd.read_csv("../../data/processed/metrics/reactome_metrics_apid_huri.csv", sep=',', header=0, index_col=0))
print(reactome.shape)
reactome

(17089, 426)


array([[1.62865015e-04, 1.43420032e-04, 1.43420032e-04, ...,
        1.82655022e-04, 1.65701943e-04, 1.65701943e-04],
       [1.00566462e-04, 5.35673183e-05, 5.35673183e-05, ...,
        9.61235563e-05, 1.29070399e-04, 1.29070399e-04],
       [1.78111514e-05, 1.48345008e-05, 1.48345008e-05, ...,
        1.76992570e-05, 2.03986618e-05, 2.03986618e-05],
       ...,
       [9.17502364e-07, 1.08097815e-06, 1.08097815e-06, ...,
        1.03499977e-06, 1.04754001e-06, 1.04754001e-06],
       [9.27519950e-07, 1.44721395e-06, 1.44721395e-06, ...,
        1.73030818e-06, 1.07996238e-06, 1.07996238e-06],
       [9.17502364e-07, 1.08097815e-06, 1.08097815e-06, ...,
        1.03499977e-06, 1.04754001e-06, 1.04754001e-06]])

In [16]:
reactome_labels = pd.read_csv('../../data/processed/reactome_labels_apid_huri.csv', header = None)
reactome_labels.index= pd.read_csv("../../data/processed/metrics/reactome_metrics_apid_huri.csv", header=0, index_col=0).index
reactome_labels.columns= pd.read_csv("../../data/processed/metrics/reactome_metrics_apid_huri.csv", header=0, index_col=0).columns
print(reactome_labels.shape)
reactome_labels.head(2)

(17089, 426)


,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
SCRIB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARHGEF7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
reactome_fs = pd.read_csv("../../data/processed/fs/reactome_apid_huri.csv")
print(reactome_fs.shape)
reactome_fs.head(2)

(426, 299)


,result.1,result.2,result.3,result.4,result.5,result.6,result.7,result.8,result.9,result.10,...,result.414,result.415,result.417,result.419,result.421,result.422,result.423,result.424,result.425,result.426
0,18.424554,0.125218,0.114776,0.165712,0.028801,0.229715,0.163529,0.128769,0.130604,0.102407,...,0.075334,0.012056,0.013855,0.029297,0.985202,0.709918,1.059249,0.975577,0.177692,0.220787
1,0.133560,7.985435,8.777931,0.057642,0.054015,0.135188,0.062448,0.084157,0.044868,0.133419,...,0.001542,0.104665,0.822205,0.070612,0.544571,0.426107,0.471597,0.407100,0.020350,0.049993


# 2. Classification Tasks

## 2.1. Complete Network

In [96]:
importlib.reload(classifiers)
classifier_results, classifier_results_proba_df, cv_results, n_fs, clf_models = classifiers.logistic_classifier(reactome, reactome_fs, reactome_labels)

  0%|          | 0/426 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
importlib.reload(classifiers)

rwr_lgr_clf, rwr_lgr_proba_clf, rwr_cv, rwr_n_fs, rwr_models = logistic_classifier(
    clf, reactome, reactome_rwr_test_index, reactome_rwr_fs_ar, reactome_labels, jobs=20)
rwr_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/rwr_lgr.csv', index=False)
rwr_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/rwr_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/rwr.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv))
with open("../../models/GAP-MINE/process/n_fs/rwr.txt", "w") as f:
    for s in rwr_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/rwr.pckl", "wb") as f:
    for model in rwr_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_clf, rwr_lgr_proba_clf, rwr_cv, rwr_n_fs, rwr_models = classifiers.multiple_fs_classifier(clf, parameters, reactome, reactome_rwr_test_index, reactome_rwr_fs_ar, reactome_labels, jobs=20)
rwr_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/rwr_lgr.csv', index=False)
rwr_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/rwr_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/rwr.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv))
with open("../../models/GAP-MINE/process/n_fs/rwr.txt", "w") as f:
    for s in rwr_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/rwr.pckl", "wb") as f:
    for model in rwr_models:
         pickle.dump(model, f)